In [27]:
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Defining vocabluary class

In [28]:
SOS_token = 0
EOS_token = 1

class LanguageVocabulary(object):
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [29]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [30]:
def read_languages(lang1, lang2, reverse=False):
    print("Reading lines...")

    lines = open('./deu.txt', encoding='utf-8').read().strip().split('\n')
    pairs = [[normalize_string(s) for s in l.split('\t')[:2]] for l in lines]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = LanguageVocabulary(lang2)
        output_lang = LanguageVocabulary(lang1)
    else:
        input_lang = LanguageVocabulary(lang1)
        output_lang = LanguageVocabulary(lang2)
    return input_lang, output_lang, pairs

In [31]:
MAX_LENGTH = 5

def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [32]:
def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_languages(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [33]:
input_lang, output_lang, pairs = prepare_data('eng', 'deu', False)
print(random.choice(pairs))

Reading lines...
Read 267186 sentence pairs
Trimmed to 11267 sentence pairs
Counting words...
Counted words:
eng 3195
deu 4322
['i lied .', 'ich habe gelogen .']


Defining encoder/decoder models

In [34]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()

        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [35]:
class AttentionDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Defining utils for models training

In [36]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [37]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

In [38]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

In [39]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [48]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)

Training models:

In [53]:
hidden_size = 512
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = AttentionDecoder(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters(encoder1, decoder1, 100000, print_every=1000)

0m 16s (- eta: 26m 24s) (1000 1%) 3.9843
0m 25s (- eta: 20m 46s) (2000 2%) 3.3634
0m 34s (- eta: 18m 47s) (3000 3%) 3.0920
0m 44s (- eta: 17m 40s) (4000 4%) 2.9390
0m 53s (- eta: 16m 58s) (5000 5%) 2.8178
1m 2s (- eta: 16m 26s) (6000 6%) 2.6980
1m 12s (- eta: 16m 2s) (7000 7%) 2.5822
1m 21s (- eta: 15m 41s) (8000 8%) 2.4756
1m 31s (- eta: 15m 22s) (9000 9%) 2.3729
1m 40s (- eta: 15m 5s) (10000 10%) 2.2359
1m 50s (- eta: 14m 50s) (11000 11%) 2.1092
1m 59s (- eta: 14m 36s) (12000 12%) 2.0820
2m 8s (- eta: 14m 22s) (13000 13%) 1.9978
2m 18s (- eta: 14m 9s) (14000 14%) 1.9152
2m 27s (- eta: 13m 56s) (15000 15%) 1.8369
2m 36s (- eta: 13m 44s) (16000 16%) 1.7731
2m 46s (- eta: 13m 32s) (17000 17%) 1.7690
2m 55s (- eta: 13m 21s) (18000 18%) 1.6685
3m 5s (- eta: 13m 11s) (19000 19%) 1.6009
3m 15s (- eta: 13m 0s) (20000 20%) 1.5250
3m 24s (- eta: 12m 49s) (21000 21%) 1.4693
3m 33s (- eta: 12m 38s) (22000 22%) 1.4199
3m 43s (- eta: 12m 27s) (23000 23%) 1.4215
3m 52s (- eta: 12m 16s) (24000 24%) 

Defining code for models evaluation

In [54]:
def translate_sentence(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for _ in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [79]:
def evaluateRandomly(encoder, decoder, n=1000):

    refs = []
    hyps = []
    for i in range(n):
        pair = random.choice(pairs)

        translated_sentence = ' '.join(translate_sentence(encoder, decoder, pair[0]))
        
        print('>', pair[0])
        print('=', pair[1])
        print('<', translated_sentence)
        print('')

        refs.append([tensorFromSentence(output_lang, pair[1]).squeeze().tolist()])
        hyps.append(tensorFromSentence(output_lang, translated_sentence).squeeze().tolist())
    
    return refs, hyps

Evaluating model:

In [80]:
refs, hyps = evaluateRandomly(encoder1, decoder1)

> tom yawned .
= tom gahnte .
< tom gahnte .

> just water please .
= nur wasser bitte .
< bitte nur wasser .

> everybody loves her .
= alle lieben sie .
< jeder liebt sie .

> i braked suddenly .
= ich bremste plotzlich .
< ich bremste plotzlich .

> you were snoring .
= du hast geschnarcht .
< du hast geschnarcht .

> don t complain .
= beschwer dich nicht .
< beschwer dich nicht .

> come here .
= kommt her !
< komm hierher .

> we need donations .
= wir brauchen spenden .
< wir brauchen spenden .

> tom fought bravely .
= tom kampfte wacker .
< tom kampfte wacker .

> don t smile .
= lachel nicht !
< lach nicht .

> tom is immoral .
= tom ist unmoralisch .
< tom ist unmoralisch .

> sign this .
= unterschreib das .
< unterschreib das .

> how tragic !
= wie tragisch !
< wie tragisch !

> where were they ?
= wo waren sie ?
< wo waren sie ?

> they adore tom .
= sie lieben tom .
< sie lieben tom .

> parties are fun .
= feiern machen spa .
< feiern machen freude .

> tom is mortifie

In [81]:
refs

[[[116, 342, 3, 1]],
 [[299, 879, 637, 3, 1]],
 [[267, 1007, 53, 3, 1]],
 [[32, 948, 3511, 3, 1]],
 [[263, 266, 2138, 3, 1]],
 [[2230, 30, 119, 3, 1]],
 [[150, 458, 5, 1]],
 [[112, 1357, 3756, 3, 1]],
 [[116, 962, 3894, 3, 1]],
 [[1065, 119, 5, 1]],
 [[116, 171, 1565, 3, 1]],
 [[647, 291, 3, 1]],
 [[310, 787, 5, 1]],
 [[1017, 1020, 53, 49, 1]],
 [[53, 1007, 116, 3, 1]],
 [[2759, 884, 305, 3, 1]],
 [[116, 171, 2691, 3, 1]],
 [[116, 2835, 3, 1]],
 [[439, 636, 621, 3, 1]],
 [[161, 53, 742, 5, 1]],
 [[116, 170, 1658, 3, 1]],
 [[53, 3630, 116, 3, 1]],
 [[116, 2487, 3, 1]],
 [[116, 171, 1160, 3, 1]],
 [[116, 171, 1565, 3, 1]],
 [[112, 1360, 3, 1]],
 [[220, 253, 5, 1]],
 [[291, 171, 373, 3, 1]],
 [[116, 1448, 3270, 3, 1]],
 [[2174, 603, 177, 49, 1]],
 [[53, 1090, 177, 3, 1]],
 [[53, 433, 3232, 3, 1]],
 [[116, 171, 2091, 3, 1]],
 [[2235, 796, 4063, 3, 1]],
 [[168, 5, 1]],
 [[53, 114, 575, 3, 1]],
 [[116, 694, 1144, 3, 1]],
 [[765, 30, 119, 5, 1]],
 [[32, 42, 862, 3, 1]],
 [[116, 476, 1230, 3, 

In [82]:
hyps

[[116, 342, 3, 1],
 [637, 299, 879, 3, 1],
 [2235, 1090, 53, 3, 1],
 [32, 948, 3511, 3, 1],
 [263, 266, 2138, 3, 1],
 [2230, 30, 119, 3, 1],
 [148, 457, 3, 1],
 [112, 1357, 3756, 3, 1],
 [116, 962, 3894, 3, 1],
 [1063, 119, 3, 1],
 [116, 171, 1565, 3, 1],
 [647, 291, 3, 1],
 [310, 787, 5, 1],
 [1017, 1020, 53, 49, 1],
 [53, 1007, 116, 3, 1],
 [2759, 884, 1424, 3, 1],
 [116, 171, 2691, 3, 1],
 [116, 2835, 3, 1],
 [439, 621, 5, 1],
 [160, 742, 5, 1],
 [116, 170, 1658, 3, 1],
 [53, 3630, 116, 3, 1],
 [116, 2487, 3, 1],
 [116, 171, 1160, 3, 1],
 [116, 171, 1565, 3, 1],
 [112, 1360, 3, 1],
 [379, 253, 253, 5, 1],
 [291, 171, 373, 3, 1],
 [116, 1448, 3270, 3, 1],
 [2175, 603, 177, 49, 1],
 [53, 1090, 177, 3, 1],
 [53, 433, 3231, 3, 1],
 [116, 171, 2091, 3, 1],
 [2235, 796, 4063, 3, 1],
 [168, 5, 1],
 [263, 114, 575, 3, 1],
 [116, 694, 1144, 3, 1],
 [765, 30, 119, 5, 1],
 [32, 42, 862, 3, 1],
 [116, 476, 1230, 3, 1],
 [876, 395, 227, 3, 1],
 [112, 433, 569, 3, 1],
 [53, 1020, 3128, 3, 1],
 [2

In [86]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

corpus_bleu_score = corpus_bleu(refs, hyps)
print("Final BLEU Score:", corpus_bleu_score)

Final BLEU Score: 0.8485211344314643


In [84]:
refs[3]

[[32, 948, 3511, 3, 1]]

In [85]:
hyps[3]

[32, 948, 3511, 3, 1]